## Homework

In this homework, we'll deploy the Bank Marketing model from the homework 5.
We already have a docker image for this model - we'll use it for 
deploying the model to Kubernetes.

## Building the image

Clone the course repo if you haven't:

```
git clone https://github.com/DataTalksClub/machine-learning-zoomcamp.git
```

Go to the `course-zoomcamp/cohorts/2024/05-deployment/homework` folder and 
execute the following:


```bash
docker build -t zoomcamp-model:3.11.5-hw10 .
```

> **Note:** If you have troubles building the image, you can 
> use the image we built and published to docker hub:
> `docker pull svizor/zoomcamp-model:3.11.5-hw10`

## Question 1

Run it to test that it's working locally:

```bash
docker run -it --rm -p 9696:9696 zoomcamp-model:3.11.5-hw10
```

And in another terminal, execute `q6_test.py` file:

```bash
python q6_test.py
```

You should see this:

```python
{'has_subscribed': True, 'has_subscribed_probability': <value>}
```

Here `<value>` is the probability of getting a subscription. You need to choose the right one.

* 0.287
* 0.530
* 0.757
* 0.960

Now you can stop the container running in Docker.

In [10]:
!ls -lh machine-learning-zoomcamp/cohorts/2024/05-deployment/homework

total 56K
-rw-rw-rw- 1 codespace codespace 253 Dec 16 07:35 Dockerfile
-rw-rw-rw- 1 codespace codespace 191 Dec 16 07:35 Pipfile
-rw-rw-rw- 1 codespace codespace 20K Dec 16 07:35 Pipfile.lock
-rw-rw-rw- 1 codespace codespace 560 Dec 16 07:35 dv.bin
-rw-rw-rw- 1 codespace codespace 850 Dec 16 07:35 model1.bin
-rw-rw-rw- 1 codespace codespace 315 Dec 16 07:35 q3_test.py
-rw-rw-rw- 1 codespace codespace 699 Dec 16 07:35 q4_predict.py
-rw-rw-rw- 1 codespace codespace 192 Dec 16 07:35 q4_test.py
-rw-rw-rw- 1 codespace codespace 699 Dec 16 07:35 q6_predict.py
-rw-rw-rw- 1 codespace codespace 195 Dec 16 07:35 q6_test.py


In [11]:
!cd machine-learning-zoomcamp/cohorts/2024/05-deployment/homework && \
 python q6_test.py

{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}


Here `<value>` is the probability of getting a subscription. You need to choose the right one.

* 0.287
* 0.530
* **0.757**
* 0.960

## Installing `kubectl` and `kind`

You need to install:

* `kubectl` - https://kubernetes.io/docs/tasks/tools/ (you might already have it - check before installing)
* `kind` - https://kind.sigs.k8s.io/docs/user/quick-start/

## Question 2

What's the version of `kind` that you have? 

Use `kind --version` to find out.


## Creating a cluster

Now let's create a cluster with `kind`:

```bash
kind create cluster
```

And check with `kubectl` that it was successfully created:

```bash
kubectl cluster-info
```

In [87]:
!kind --version

kind version 0.25.0


## Question 3

What's the smallest deployable computing unit that we can create and manage 
in Kubernetes (`kind` in our case)?

* Node
* **Pod**
* Deployment
* Service


## Question 4

Now let's test if everything works. Use `kubectl` to get the list of running services.

What's the `Type` of the service that is already running there?

* NodePort
* **ClusterIP**
* ExternalName
* LoadBalancer

In [15]:
!kubectl get svc

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   6m9s


## Question 5

To be able to use the docker image we previously created (`zoomcamp-model:3.11.5-hw10`),
we need to register it with `kind`.

What's the command we need to run for that?

* `kind create cluster`
* `kind build node-image`
* `kind load docker-image`
* `kubectl apply`


In [16]:
!kind load docker-image zoomcamp-model:3.11.5-hw10

Image: "zoomcamp-model:3.11.5-hw10" with ID "sha256:46219213d01694a1fc8109d976fa29f8ca99a59b3c2f9586e2536dee87729003" not yet present on node "kind-control-plane", loading...


In [18]:
#!kubectl describe node kind-worker | grep -i image

Error from server (NotFound): nodes "kind-worker" not found


## Question 6

Now let's create a deployment config (e.g. `deployment.yaml`):

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: subscription
spec:
  selector:
    matchLabels:
      app: subscription
  replicas: 1
  template:
    metadata:
      labels:
        app: subscription
    spec:
      containers:
      - name: subscription
        image: <Image>
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: <Memory>
            cpu: <CPU>
        ports:
        - containerPort: <Port>
```

Replace `<Image>`, `<Memory>`, `<CPU>`, `<Port>` with the correct values.

What is the value for `<Port>`?

Apply this deployment using the appropriate command and get a list of running Pods. 
You can see one running Pod.

In [22]:
%%writefile deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: subscription
spec:
  selector:
    matchLabels:
      app: subscription
  replicas: 1
  template:
    metadata:
      labels:
        app: subscription
    spec:
      containers:
      - name: subscription
        image: zoomcamp-model:3.11.5-hw10
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: "128Mi"   
            cpu: "200m"       
        ports:
        - containerPort: 9696  

Writing deployment.yaml


In [27]:
!ls -lh deployment.yaml

-rw-rw-rw- 1 codespace codespace 533 Dec 16 14:12 deployment.yaml


In [28]:
!kubectl apply -f deployment.yaml

deployment.apps/subscription created


In [30]:
!kubectl get deploy,pod

NAME                           READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/subscription   1/1     1            1           6s

NAME                                READY   STATUS    RESTARTS   AGE
pod/subscription-544b4f9664-j52nd   1/1     Running   0          6s


## Question 7

Let's create a service for this deployment (`service.yaml`):

```yaml
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <???>
  ports:
  - port: 80
    targetPort: <PORT>
```

Fill it in. What do we need to write instead of `<???>`?

Apply this config file.


In [24]:
%%writefile service.yaml
apiVersion: v1
kind: Service
metadata:
  name: subscription
spec:
  type: LoadBalancer
  selector:
    app: subscription
  ports:
  - port: 80
    targetPort: 9696

Writing service.yaml


In [31]:
!ls -lh service.yaml

-rw-rw-rw- 1 codespace codespace 164 Dec 16 14:12 service.yaml


In [32]:
!kubectl apply -f service.yaml

service/subscription created


In [33]:
!kubectl get svc

NAME           TYPE           CLUSTER-IP   EXTERNAL-IP   PORT(S)        AGE
kubernetes     ClusterIP      10.96.0.1    <none>        443/TCP        17m
subscription   LoadBalancer   10.96.75.0   <pending>     80:31260/TCP   13s


In [36]:
!kubectl get po -o wide

NAME                            READY   STATUS    RESTARTS   AGE     IP           NODE                 NOMINATED NODE   READINESS GATES
subscription-544b4f9664-j52nd   1/1     Running   0          2m43s   10.244.0.5   kind-control-plane   <none>           <none>


In [34]:
!kubectl describe svc subscription

Name:                     subscription
Namespace:                default
Labels:                   <none>
Annotations:              <none>
Selector:                 app=subscription
Type:                     LoadBalancer
IP Family Policy:         SingleStack
IP Families:              IPv4
IP:                       10.96.75.0
IPs:                      10.96.75.0
Port:                     <unset>  80/TCP
TargetPort:               9696/TCP
NodePort:                 <unset>  31260/TCP
Endpoints:                10.244.0.5:9696
Session Affinity:         None
External Traffic Policy:  Cluster
Internal Traffic Policy:  Cluster
Events:                   <none>


## Testing the service

We can test our service locally by forwarding the port 9696 on our computer 
to the port 80 on the service:

```bash
kubectl port-forward service/<Service name> 9696:80
```

Run `q6_test.py` (from the homework 5) once again to verify that everything is working. 
You should get the same result as in Question 1.

In [40]:
#!kubectl port-forward service/subscription 9696:80

In [42]:
!cd machine-learning-zoomcamp/cohorts/2024/05-deployment/homework && \
 python q6_test.py

{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}


## Autoscaling

Now we're going to use a [HorizontalPodAutoscaler](https://kubernetes.io/docs/tasks/run-application/horizontal-pod-autoscale-walkthrough/) 
(HPA for short) that automatically updates a workload resource (such as our deployment), 
with the aim of automatically scaling the workload to match demand.

Use the following command to create the HPA:

```bash
kubectl autoscale deployment subscription --name subscription-hpa --cpu-percent=20 --min=1 --max=3
```

You can check the current status of the new HPA by running:

```bash
kubectl get hpa
```

The output should be similar to the next:

```bash
NAME               REFERENCE                 TARGETS   MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   1%/20%    1         3         1          27s
```

`TARGET` column shows the average CPU consumption across all the Pods controlled by the corresponding deployment.
Current CPU consumption is about 0% as there are no clients sending requests to the server.
> 
>Note: In case the HPA instance doesn't run properly, try to install the latest Metrics Server release 
> from the `components.yaml` manifest:
> ```bash
> kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml
>```

In [47]:
!kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml

serviceaccount/metrics-server created
clusterrole.rbac.authorization.k8s.io/system:aggregated-metrics-reader created
clusterrole.rbac.authorization.k8s.io/system:metrics-server created
rolebinding.rbac.authorization.k8s.io/metrics-server-auth-reader created
clusterrolebinding.rbac.authorization.k8s.io/metrics-server:system:auth-delegator created
clusterrolebinding.rbac.authorization.k8s.io/system:metrics-server created
service/metrics-server created
deployment.apps/metrics-server created
apiservice.apiregistration.k8s.io/v1beta1.metrics.k8s.io created


In [79]:
!kubectl get deployment metrics-server -n kube-system

NAME             READY   UP-TO-DATE   AVAILABLE   AGE
metrics-server   1/1     1            1           33m


In [73]:
!kubectl logs -n kube-system deployment/metrics-server

I1216 14:55:42.011340       1 serving.go:374] Generated self-signed cert (/tmp/apiserver.crt, /tmp/apiserver.key)
I1216 14:55:42.506082       1 handler.go:275] Adding GroupVersion metrics.k8s.io v1beta1 to ResourceManager
I1216 14:55:42.614378       1 secure_serving.go:213] Serving securely on [::]:10250
I1216 14:55:42.614449       1 requestheader_controller.go:169] Starting RequestHeaderAuthRequestController
I1216 14:55:42.614459       1 shared_informer.go:311] Waiting for caches to sync for RequestHeaderAuthRequestController
I1216 14:55:42.614493       1 dynamic_serving_content.go:132] "Starting controller" name="serving-cert::/tmp/apiserver.crt::/tmp/apiserver.key"
I1216 14:55:42.614599       1 tlsconfig.go:240] "Starting DynamicServingCertificateController"
I1216 14:55:42.614709       1 configmap_cafile_content.go:202] "Starting controller" name="client-ca::kube-system::extension-apiserver-authentication::client-ca-file"
I1216 14:55:42.614732       1 shared_informer.go:311] Waiting

If existe following fail:

"Failed to scrape node" err="Get \"https://172.18.0.2:10250/metrics/resource\": tls: failed to verify certificate: x509: cannot validate certificate for 172.18.0.2 because it doesn't contain any IP SANs" node="kind-control-plane"


```bash
kubectl edit deployment metrics-server -n kube-system
```

```yaml
spec:
  containers:
  - name: metrics-server
    image: k8s.gcr.io/metrics-server/metrics-server:v0.6.1
    args:
      - --cert-dir=/tmp
      - --secure-port=4443
      - --kubelet-insecure-tls   # Añadir esta línea
      - --kubelet-preferred-address-types=InternalIP,ExternalIP,Hostname
```

In [75]:
#!kubectl delete hpa subscription-hpa

In [76]:
!kubectl autoscale deployment subscription --name subscription-hpa --cpu-percent=20 --min=1 --max=3

horizontalpodautoscaler.autoscaling/subscription-hpa autoscaled


In [85]:
!kubectl get hpa

NAME               REFERENCE                 TARGETS       MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   cpu: 1%/20%   1         3         1          5m58s


## Increase the load

Let's see how the autoscaler reacts to increasing the load. To do this, we can slightly modify the existing
`q6_test.py` script by putting the operator that sends the request to the subscription service into a loop.

```python
while True:
    sleep(0.1)
    response = requests.post(url, json=client).json()
    print(response)
```

Now you can run this script.

In [82]:
%%writefile q6_test_mod.py
import requests
from time import sleep
url = "http://localhost:9696/predict"
client = {"job": "management", "duration": 400, "poutcome": "success"}
while True:
    sleep(0.1)
    response = requests.post(url, json=client).json()
    print(response)

Overwriting q6_test_mod.py


## Question 8 (optional)

Run `kubectl get hpa subscription-hpa --watch` command to monitor how the autoscaler performs. 
Within a minute or so, you should see the higher CPU load; and then - more replicas. 
What was the maximum amount of the replicas during this test?


* 1
* 2
* **3**
* 4

> Note: It may take a few minutes to stabilize the number of replicas. Since the amount of load is not controlled 
> in any way it may happen that the final number of replicas will differ from initial.

In [86]:
!kubectl get hpa subscription-hpa --watch

NAME               REFERENCE                 TARGETS       MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   cpu: 1%/20%   1         3         1          6m3s
subscription-hpa   Deployment/subscription   cpu: 11%/20%   1         3         1          7m46s
subscription-hpa   Deployment/subscription   cpu: 14%/20%   1         3         1          8m1s
subscription-hpa   Deployment/subscription   cpu: 15%/20%   1         3         1          8m46s
subscription-hpa   Deployment/subscription   cpu: 14%/20%   1         3         1          9m1s
subscription-hpa   Deployment/subscription   cpu: 13%/20%   1         3         1          11m
subscription-hpa   Deployment/subscription   cpu: 14%/20%   1         3         1          11m
subscription-hpa   Deployment/subscription   cpu: 13%/20%   1         3         1          11m
subscription-hpa   Deployment/subscription   cpu: 14%/20%   1         3         1          11m
subscription-hpa   Deployment/subscription   

## Submit the results

* Submit your results here: https://courses.datatalks.club/ml-zoomcamp-2024/homework/hw10
* If your answer doesn't match options exactly, select the closest one